# mAP

Compute mAP and all related values to it based upon [this](https://medium.com/@jonathan_hui/map-mean-average-precision-for-object-detection-45c121a31173) blog. Related values are:

- precision
- recall
- average precision

In [62]:
import numpy as np
import tensorflow as tf

In [ ]:
tf.global_variables_initializer

In [2]:
X = np.array([
    [1, True],
    [2, True],
    [3, False],
    [4, False],
    [5, False],
    [6, True],
    [7, True],
    [8, False],
    [9, False],
    [10, True]
])
X

array([[ 1,  1],
       [ 2,  1],
       [ 3,  0],
       [ 4,  0],
       [ 5,  0],
       [ 6,  1],
       [ 7,  1],
       [ 8,  0],
       [ 9,  0],
       [10,  1]])

In [71]:
# compute precision array
arr = []
count = 0
for idx, x in enumerate(X):
    if x[1] == 1:
        count += 1.
    arr.append(count / (idx+1))
precision_arr = np.array(arr)
precision_arr

array([1.        , 1.        , 0.66666667, 0.5       , 0.4       ,
       0.5       , 0.57142857, 0.5       , 0.44444444, 0.5       ])

In [73]:
import pprint

pprint.pprint(list(precision_arr))

[1.0,
 1.0,
 0.6666666666666666,
 0.5,
 0.4,
 0.5,
 0.5714285714285714,
 0.5,
 0.4444444444444444,
 0.5]


In [4]:
# compute recall array
pos_labels = 5
arr = []
count = 0
for x in X:
    if x[1] == 1:
        count += 1.
    arr.append(count / pos_labels)
recall_arr = np.array(arr)
recall_arr

array([0.2, 0.4, 0.4, 0.4, 0.4, 0.6, 0.8, 0.8, 0.8, 1. ])

In [5]:
# combine all data
x2 = np.array([X[:,0], X[:,1], precision_arr, recall_arr])
x2

array([[ 1.        ,  2.        ,  3.        ,  4.        ,  5.        ,
         6.        ,  7.        ,  8.        ,  9.        , 10.        ],
       [ 1.        ,  1.        ,  0.        ,  0.        ,  0.        ,
         1.        ,  1.        ,  0.        ,  0.        ,  1.        ],
       [ 1.        ,  1.        ,  0.66666667,  0.5       ,  0.4       ,
         0.5       ,  0.57142857,  0.5       ,  0.44444444,  0.5       ],
       [ 0.2       ,  0.4       ,  0.4       ,  0.4       ,  0.4       ,
         0.6       ,  0.8       ,  0.8       ,  0.8       ,  1.        ]])

In [6]:
# average precision
prev_max = None
arr = []
bins = np.arange(0.0, 1.1, .1)
for idx, i in enumerate(reversed(bins)):
    try:
        prev_max = np.where(x2[3, :]==i)[0][0]
    except IndexError:
        pass
    arr.append(precision_arr[prev_max])
avg_prec_arr = np.array([x for x in reversed(arr)])
avg_prec_arr

array([1.        , 1.        , 1.        , 1.        , 1.        ,
       0.57142857, 0.57142857, 0.57142857, 0.57142857, 0.5       ,
       0.5       ])

In [63]:
# mAP
np_ap = np.sum(avg_prec_arr) / len(avg_prec_arr)
np_ap

0.7532467532467532

In [11]:
sess = tf.InteractiveSession()

In [68]:
recall = tf.constant(recall_arr)
precision = tf.constant(precision_arr)

l_aps = []
for t in np.arange(0., 1.1, 0.1):
    mask = tf.greater_equal(recall, t)
    v = tf.reduce_max(tf.boolean_mask(precision, mask))
    l_aps.append(v / 11.)
ap = tf.add_n(l_aps)

tf_ap = sess.run(ap)
tf_ap

0.7532467532467533

In [74]:
np.allclose([np_ap], [tf_ap])

True

In [75]:
c = tf.constant([1], dtype=tf.float64)
sess.run(c)

array([1.])

In [70]:
def average_precision_voc07(precision, recall, name=None):
    """Compute (interpolated) average precision from precision and recall Tensors.

    The implementation follows Pascal 2007 guidelines.
    See also: https://sanchom.wordpress.com/tag/average-precision/
    """
    with tf.name_scope(name, 'average_precision_voc07', [precision, recall]):
        # Convert to float64 to decrease error on cumulated sums.
        precision = tf.cast(precision, dtype=tf.float64)
        recall = tf.cast(recall, dtype=tf.float64)
        # Add zero-limit value to avoid any boundary problem...
        precision = tf.concat([precision, [0.]], axis=0)
        recall = tf.concat([recall, [np.inf]], axis=0)

        # Split the integral into 10 bins.
        l_aps = []
        for t in np.arange(0., 1.1, 0.1):
            mask = tf.greater_equal(recall, t)
            v = tf.reduce_max(tf.boolean_mask(precision, mask))
            l_aps.append(v / 11.)
        ap = tf.add_n(l_aps)
        return ap
    
recall = tf.constant(recall_arr)
precision = tf.constant(precision_arr)

sess.run(average_precision_voc07(precision, recall))

0.7532467532467533

In [ ]:
t = tf.constant(t, dtype=tf.float64)
mask = tf.greater_equal(recall, t)
sess.run(mask)

In [61]:
# together
t = np.mgrid[0.0:1.1:.1, 0.0:1.:.1][0]
t = tf.constant(t, dtype=tf.float64)
mask = tf.greater_equal(recall, t)
precision = tf.reshape(tf.tile(precision, tf.constant([11])), shape=(10, 11))
v = tf.reduce_max(tf.boolean_mask(precision, tf.transpose(mask)))
sess.run(v)

1.0

In [60]:
tf.reshape(tf.tile(precision, tf.constant([11])), shape=(10, 11))

<tf.Tensor 'Reshape_1:0' shape=(10, 11) dtype=float64>

In [55]:
sess.run(tf.constant([1])).shape

(1,)

In [27]:
np.arange(0., 1.1, 0.1)

array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ])

In [28]:
len(np.arange(0., 1.1, 0.1))

11

In [23]:
recall = tf.constant(recall_arr)
t = .4
mask = tf.greater_equal(recall, t)
print(len(sess.run(mask)))
sess.run(mask)

10


array([False,  True,  True,  True,  True,  True,  True,  True,  True,
        True])

In [17]:
bool_mask = tf.boolean_mask(precision, mask)
sess.run(bool_mask)

array([1.        , 0.66666667, 0.5       , 0.4       , 0.5       ,
       0.57142857, 0.5       , 0.44444444, 0.5       ])

In [18]:
rmax = tf.reduce_max(bool_mask)
sess.run(rmax)

1.0

In [19]:
sess.run(recall)

array([0.2, 0.4, 0.4, 0.4, 0.4, 0.6, 0.8, 0.8, 0.8, 1. ])

In [20]:
sess.run(precision)

array([1.        , 1.        , 0.66666667, 0.5       , 0.4       ,
       0.5       , 0.57142857, 0.5       , 0.44444444, 0.5       ])